<a href="https://colab.research.google.com/github/Dhanasree-Rajamani/Data-Mining/blob/main/Data%20Mining%20Assignment%205/Product_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and Import Statements

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 96 kB/s 


In [ ]:
import pickle

In [ ]:
import faiss

Load dataset from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_data():
    with open('/content/drive/MyDrive/DataMining_Datasets/movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

In [ ]:
import numpy as np

In [ ]:
np.where(data['name'] == 'Fifth Element, The (1997)')

(array([249]),)

In [ ]:
data['vector'][0]

array([-0.01780608, -0.14265831,  0.10308606, -0.41564542,  0.13982998,
       -0.16974038,  0.15737218,  0.04518698, -0.10058694, -0.02617188,
        0.0406148 , -0.14783277, -0.07933554,  0.20399824, -0.51070064,
       -0.30125597, -0.12879208, -0.09246405, -0.3589764 , -0.02050611,
        0.12336025,  0.14336675,  0.19412877, -0.08913906, -0.16755173,
        0.2030852 , -0.24195394, -0.27702233, -0.37455535,  0.1277788 ,
       -0.3171818 , -0.17515987, -0.16746472, -0.04281802,  0.02517505,
        0.2907528 , -0.23536316, -0.03609738,  0.33521986,  0.02485491,
       -0.30672348, -0.22360437,  0.1108979 ,  0.12440509, -0.14316128,
       -0.0903152 , -0.10247427, -0.08632833, -0.05165194,  0.10785805,
        0.10544457,  0.35192525, -0.04853973, -0.04199679,  0.5817107 ,
        0.02434268,  0.02575092,  0.25519538, -0.0878452 ,  0.10958093,
        0.20129448,  0.09659795, -0.17529577, -0.03061521], dtype=float32)

Implementing Product quantization with faiss:

Vector is represented by 8 sub-vectors, Inverted index are created for efficient searching

In [ ]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

  Give input to search and return result using Product Quantization for finding similar items

In [ ]:
movie_index = 249
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most similar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most similar movies to Fifth Element, The (1997) are:


['Fifth Element, The (1997)',
 'Men in Black (1997)',
 'Breakdown (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Con Air (1997)',
 'Chasing Amy (1997)',
 'Private Parts (1997)',
 'Austin Powers: International Man of Mystery (1997)',
 'Face/Off (1997)',
 'Grosse Pointe Blank (1997)']